In [ ]:
import os
import azure.ai.vision as sdk

subscription_key = "VISON_KEY" #vison key
endpoint = "VISION_ENDPOINT" #vision endpoint


In [ ]:
from PIL import Image, ImageDraw

import requests
from io import BytesIO
from IPython.display import display


read_image_url = "https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png"

# Get the image from the URL
response = requests.get(read_image_url)

# Open the image using PIL
img = Image.open(BytesIO(response.content))

# Display the image
display(img)

In [ ]:
service_options = sdk.VisionServiceOptions(endpoint,
                                           subscription_key)

vision_source = sdk.VisionSource(
    url=read_image_url)

analysis_options = sdk.ImageAnalysisOptions()

analysis_options.features = (
    sdk.ImageAnalysisFeature.CAPTION |
    sdk.ImageAnalysisFeature.OBJECTS |
    sdk.ImageAnalysisFeature.DENSE_CAPTIONS |
    sdk.ImageAnalysisFeature.TAGS |
    sdk.ImageAnalysisFeature.CROP_SUGGESTIONS |
    sdk.ImageAnalysisFeature.PEOPLE
    #sdk.ImageAnalysisFeature.TEXT
)

analysis_options.language = "en"

analysis_options.gender_neutral_caption = True

image_analyzer = sdk.ImageAnalyzer(service_options, vision_source, analysis_options)

result = image_analyzer.analyze()

if result.reason == sdk.ImageAnalysisResultReason.ANALYZED:

    if result.caption is not None:
        print(" Caption:")
        print("   '{}', Confidence {:.4f}".format(result.caption.content, result.caption.confidence))

    if result.text is not None:
        print(" Text:")
        for line in result.text.lines:
            points_string = "{" + ", ".join([str(int(point)) for point in line.bounding_polygon]) + "}"
            print("   Line: '{}', Bounding polygon {}".format(line.content, points_string))
            for word in line.words:
                points_string = "{" + ", ".join([str(int(point)) for point in word.bounding_polygon]) + "}"
                print("     Word: '{}', Bounding polygon {}, Confidence {:.4f}"
                      .format(word.content, points_string, word.confidence))
                
    if result.dense_captions is not None:
        print(" dense Captions:")
        for caption in result.dense_captions:
            print("   Line: '{}', Confidence: {}, Bounding box [{}]".format(caption.content, caption.confidence, caption.bounding_box))

    if result.objects is not None:
        print(" Objects:")
        for object_detected in result.objects:
            print("   Object: '{}', Confidence: {}, Bounding box [{}]".format(object_detected.name, object_detected.confidence, object_detected.bounding_box))


    if result.people is not None:
        print(" People:")
        for person in result.people:
            print("   Confidence: {}, Bounding box [{}]".format(person.confidence, person.bounding_box))
        
    if result.tags is not None:
        print(" Tags:")
        for tag in result.tags:
            print("   Tag: '{}', Confidence: {}".format(tag.name, tag.confidence))
        
    if result.crop_suggestions is not None:
        print(" Crop Suggestions:")
        for region in result.crop_suggestions:
            print("   Crop suggestion: {}".format(region.bounding_box))
else:

    error_details = sdk.ImageAnalysisErrorDetails.from_result(result)
    print(" Analysis failed.")
    print("   Error reason: {}".format(error_details.reason))
    print("   Error code: {}".format(error_details.error_code))
    print("   Error message: {}".format(error_details.message))